<a href="https://colab.research.google.com/github/salimahdiyyah/Dashboard/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from babel.numbers import format_currency
sns.set(style='dark')

In [133]:
def create_monthly_orders_df(df):
    monthly_orders_df = df.resample(rule='M', on='order_purchase_timestamp').agg({
        "order_id": "nunique",
        "payment_value": "sum"
    })
    monthly_orders_df = monthly_orders_df.reset_index()
    monthly_orders_df.rename(columns={
        "order_id": "order_count",
        "payment_value": "revenue"
    }, inplace=True)

    return monthly_orders_df

In [134]:
def create_sum_items_df(df):
    sum_items_df = df.groupby("product_category_name_english").order_id.nunique().sort_values(ascending=False).reset_index()
    sum_items_df.rename(columns={"product_category_name_english": "product_category", "order_id":"count"}, inplace=True)
    return sum_items_df

In [135]:
def create_product_by_payment(df):
    product_by_payment = df.groupby("product_category_name_english").payment_value.nunique().sort_values(ascending=False).reset_index()
    product_by_payment.rename(columns={"product_category_name_english": "product_category", "payment_value":"count"}, inplace=True)
    return product_by_payment

In [136]:
def create_bycity_df(df):
    bycity_df = df.groupby(by="customer_city").agg({"customer_id":"nunique", "payment_value":"sum"}).sort_values(ascending=False, by="payment_value").reset_index()
    bycity_df.rename(columns={
        "customer_id": "count",
        "customer_city":"city",
        "payment_value":"revenue"
    }, inplace=True)

    return bycity_df

In [137]:
def create_rfm_df(df):
    rfm_df = df.groupby(by="customer_unique_id", as_index=False).agg({
        "order_purchase_timestamp": "max", #mengambil tanggal order terakhir
        "order_id": "nunique",
        "payment_value": "sum"
    })
    rfm_df.columns = ["customer_unique_id", "max_order_timestamp", "frequency", "monetary"]

    # menghitung kapan terakhir pelanggan melakukan transaksi (hari)
    rfm_df["max_order_timestamp"] = rfm_df["max_order_timestamp"].dt.date
    recent_date = df["order_purchase_timestamp"].dt.date.max()
    rfm_df["recency"] = rfm_df["max_order_timestamp"].apply(lambda x: (recent_date - x).days)

    return rfm_df

In [138]:
all_df = pd.read_csv("merge_data.csv")

In [139]:
datetime_columns = ["order_purchase_timestamp", "order_estimated_delivery_date"]
all_df.sort_values(by="order_purchase_timestamp", inplace=True)
all_df.reset_index(inplace=True)

for column in datetime_columns:
    all_df[column] = pd.to_datetime(all_df[column])


In [140]:
min_date = all_df["order_purchase_timestamp"].min()
max_date = all_df["order_purchase_timestamp"].max()

with st.sidebar:
    # Menambahkan logo perusahaan
    st.image("https://raw.githubusercontent.com/salimahdiyyah/Dashboard/main/E-commerce-preview.png")

    # Mengambil start_date & end_date dari date_input
    start_date, end_date = st.date_input(
        label='Rentang Waktu',min_value=min_date,
        max_value=max_date,
        value=[min_date, max_date]
    )

In [141]:
main_df = all_df[(all_df["order_purchase_timestamp"] >= str(start_date)) &
                (all_df["order_purchase_timestamp"] <= str(end_date))]

In [142]:
monthly_orders_df = create_monthly_orders_df(main_df)
sum_items_df = create_sum_items_df(main_df)
product_by_payment = create_product_by_payment(main_df)
bycity_df = create_bycity_df(main_df)
rfm_df = create_rfm_df(main_df)

In [143]:
st.header('E-Commerce Public Dashboard :sparkles:')

DeltaGenerator()

In [144]:
st.subheader('monthly Orders')

col1, col2 = st.columns(2)

with col1:
    total_orders = monthly_orders_df.order_count.sum()
    st.metric("Total orders", value=total_orders)

with col2:
    total_revenue = format_currency(monthly_orders_df.revenue.sum(), "AUD", locale='es_CO')
    st.metric("Total Revenue", value=total_revenue)

fig, ax = plt.subplots(figsize=(16, 8))
ax.plot(
    monthly_orders_df["order_purchase_timestamp"],
    monthly_orders_df["order_count"],
    marker='o',
    linewidth=2,
    color="#90CAF9"
)
ax.tick_params(axis='y', labelsize=20)
ax.tick_params(axis='x', labelsize=15)

st.pyplot(fig)

DeltaGenerator()

In [145]:
st.subheader("Best & Worst Performing Product")

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(35, 15))

colors = ["#90CAF9", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3"]

sns.barplot(x="count", y="product_category", data=sum_items_df.head(5), palette=colors, ax=ax[0])
ax[0].set_ylabel(None)
ax[0].set_xlabel("Number of Sales", fontsize=30)
ax[0].set_title("Top 5 Best Performing Product", loc="center", fontsize=50)
ax[0].tick_params(axis='y', labelsize=35)
ax[0].tick_params(axis='x', labelsize=30)

sns.barplot(x="count", y="product_category", data=sum_items_df.sort_values(by="count", ascending=True).head(5), palette=colors, ax=ax[1])
ax[1].set_ylabel(None)
ax[1].set_xlabel("Number of Sales", fontsize=30)
ax[1].invert_xaxis()
ax[1].yaxis.set_label_position("right")
ax[1].yaxis.tick_right()
ax[1].set_title(" Top 5 Worst Performing Product", loc="center", fontsize=50)
ax[1].tick_params(axis='y', labelsize=35)
ax[1].tick_params(axis='x', labelsize=30)

st.pyplot(fig)

DeltaGenerator()

In [146]:
st.subheader("Best Performing Product Categories by Number of Revenue")

plt.figure(figsize=(10, 5))
sns.barplot(x="count", y="product_category", data=product_by_payment.head(5), palette=colors)
plt.ylabel(None)
plt.xlabel("Number of Sales", fontsize=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=10)
plt.tight_layout()

st.pyplot(fig)

DeltaGenerator()

In [147]:
# Revenue demographic
st.subheader("Revenue Demographics")
fig= plt.figure (figsize=(20, 10))
colors = ["#748E63", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3"]
sns.barplot(
    x="revenue",
    y="city",
    data=bycity_df.head(),
    palette=colors
)
plt.title("Top 5 Number of Revenue by City", loc="center", fontsize=15)
plt.ylabel(None)
plt.xlabel(None)
plt.tick_params(axis='y', labelsize=12)
st.pyplot(fig)

DeltaGenerator()

In [148]:
# Payment Method
st.subheader("Total Order by Payment Method")

plt.figure(figsize=(10,6))
colors = ["#748E63", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3"]
sns.countplot(data=all_df, x="payment_type", palette=colors)
plt.xlabel(None)
plt.ylabel(None)
plt.tight_layout()

st.pyplot(fig)

DeltaGenerator()

In [149]:
# Review Score
st.subheader("Review Score Distribution")

fig, ax = plt.subplots(figsize=(10,6))
colors = ["#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#748E63"]
sns.countplot(data=all_df, x="review_score", palette=colors, ax=ax)
ax.set_xlabel('Review Score',  fontdict={'fontsize':15})
ax.set_ylabel('Count', fontdict={'fontsize':15})
fig.tight_layout()

st.pyplot(fig)

DeltaGenerator()

In [150]:
st.subheader("Best Customer Based on RFM Parameters")

col1, col2, col3 = st.columns(3)

with col1:
    avg_recency = round(rfm_df.recency.mean(), 1)
    st.metric("Average Recency (days)", value=avg_recency)

with col2:
    avg_frequency = round(rfm_df.frequency.mean(), 2)
    st.metric("Average Frequency", value=avg_frequency)

with col3:
    avg_frequency = format_currency(rfm_df.monetary.mean(), "AUD", locale='es_CO')
    st.metric("Average Monetary", value=avg_frequency)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(35, 15))
colors = ["#90CAF9", "#90CAF9", "#90CAF9", "#90CAF9", "#90CAF9"]

sns.barplot(y="recency", x="customer_unique_id", data=rfm_df.sort_values(by="recency", ascending=True).head(5), palette=colors, ax=ax[0])
ax[0].set_ylabel(None)
ax[0].set_xlabel("customer_unique_id", fontsize=30)
ax[0].set_title("By Recency (days)", loc="center", fontsize=50)
ax[0].tick_params(axis='y', labelsize=30)
ax[0].tick_params(axis='x', labelsize=35)

sns.barplot(y="frequency", x="customer_unique_id", data=rfm_df.sort_values(by="frequency", ascending=False).head(5), palette=colors, ax=ax[1])
ax[1].set_ylabel(None)
ax[1].set_xlabel("customer_unique_id", fontsize=30)
ax[1].set_title("By Frequency", loc="center", fontsize=50)
ax[1].tick_params(axis='y', labelsize=30)
ax[1].tick_params(axis='x', labelsize=35)

sns.barplot(y="monetary", x="customer_unique_id", data=rfm_df.sort_values(by="monetary", ascending=False).head(5), palette=colors, ax=ax[2])
ax[2].set_ylabel(None)
ax[2].set_xlabel("customer_unique_id", fontsize=30)
ax[2].set_title("By Monetary", loc="center", fontsize=50)
ax[2].tick_params(axis='y', labelsize=30)
ax[2].tick_params(axis='x', labelsize=35)

st.pyplot(fig)

st.caption('Copyright (c) Salimah Dicoding 2023')

DeltaGenerator()